# GTKY: Get To Know You, The Dataset

We use IMDB review dataset from [TFDS](https://www.tensorflow.org/datasets/catalog/imdb_reviews) in these experiment. It consists of train data and test data, each of which are IMDB reviews in plain, uncleaned text format and their labels (positive and negative). 

Most data scientist miss this very important GTKY step. They just download the data and rush into building models. However, when you do empirical study, it is very important to know the data. You may come up with intuitions from just a glance.

We can also do some clean up and preprocessing before building any models. While it's always good practice to so, you should not spend too much time. Let Machine Learning do their job as well. If you have to input very clean data set to a model so that it can give you good results, then it's by no means a good model.

In [1]:
%load_ext autoreload
%autoreload

from lib.dataset import download_tfds_imdb_as_text, download_tfds_imdb_as_text_tiny
from lib.nlp_utils import spacy_tokenizer, remove_html_tags, remove_some_symbols, print_words_freq_stat

from collections import Counter

In [2]:
X_train, X_test, y_train, y_test  = download_tfds_imdb_as_text() 

# What does raw the data look like?

Let's look at high level statistics of dataset and have a glance on one sample of each class

In [3]:
print("number of train data", len(X_train) )
print("number of classes", Counter(y_train) )
print("number of train data", len(X_test) )
print("number of classes", Counter(y_test) )
print()
print("Positive Example:")
n = 100
print(y_train[n], X_train[n])
print()
print("Negative Example:")
n = 1407
print(y_train[n], X_train[n])

number of train data 25000
number of classes Counter({0: 12500, 1: 12500})
number of train data 25000
number of classes Counter({1: 12500, 0: 12500})

Positive Example:
1 I enjoyed this movie,and after watching it,it made me wonder just how many 'Caitlin Rose's' exist in the world.How many other girls have been subjected to this sort of sexual abuse,and torment by classmates and have been too frightened to open their mouth about it? Just how threatening and cruel can teenagers be towards one another,because as this film demonstrates,who's right is not foremost important,its who is popular,and feared which manipulates the minds of youths,and influences them to allow this sort of immorality to happen.Tiffani Amber Thiessen gives a powerful performance as the rape victim,and Brian Austin Green is convincing as the guy torn between the girl he thought he loved,and his best friend.This is the kind of film that doesn't get the exposure it deserves.Remarkable,and brilliant,too good to be just

Here we know that we have equal number of train and test data. We also have balance dataset i.e. equal number of samples in both classes. We also briefly get some sense of how positive and negative samples look like. 

Now let's take a look to a text sample. We can see that;

- ```happen.Tiffani``` - If you are familiar with SpaCy, you will notice that this could create tokenization problem.
- ```.<br /><br />3/10.``` - These HTML tags could also mess up tokenization. 

These are some issues we can address.



# Take a closer look on the text...

Let's try tokenize it and see what tokens look like.



In [4]:
tokenized_sents = spacy_tokenizer(X_train, do_preprocess=False)
print_words_freq_stat(tokenized_sents)


number of unique tokens 121415
number of tokens occuring only once 57868
fraction of tokens occuring only once: 0.48
least 50 common words and their occurrenceL:
[('Particulars', 1), ('Svankmajer', 1), ('Ruka', 1), ('sculpt', 1), ('metacinema', 1), ('Paradice', 1), ('Johnathin', 1), ('invlove', 1), ('/>Jules', 1), ('9/11-related', 1), ('sibling?<br', 1), ('PIggy', 1), ('7.6/10', 1), ('Commissar', 1), ('polution', 1), ('employes', 1), ('mislead.<br', 1), ('/>Google', 1), ('inflight', 1), ('MacAdams', 1), ("sensitivity'.<br", 1), ('ubiqutous', 1), ('concept!<br', 1), ('Aquatania', 1), ('décors', 1), ('Balu', 1), ('Amazons', 1), ('BESIDES', 1), ('Laughtracks', 1), ('Development).<br', 1), ('Benecio', 1), ('screening----talk', 1), ("Argentine',this", 1), ('/>THey', 1), ('Horror_Fan', 1), ('Mandatory', 1), ('VACATE', 1), ('BUILDING', 1), ('scareless', 1), ('positives-', 1), ('Snow.<br', 1), ('4-D', 1), ('spacewalk', 1), ('moonwalk', 1), ('Mayble', 1), ('Riget3', 1), ('dreams?<br', 1), ('exp

Now you can see that about 50% of unique tokens appear only once, which means that some features will have only one example during the training. This is not ideal scenario for any machine learning models since it creates the sparseness in feature space. However, this is not neccessary the case. It also depends on how we generate features out of text. We will discuss this point in later experiments. 

You can also see how HTML tags mess up the tokenization and that makes the fraction of tokens occuring only once very high. Let's remove those tags and see how much it is improved.

In [5]:
sents_remove_html = [remove_html_tags(sent) for sent in X_train]
tokenized_sents_remove_html = spacy_tokenizer(sents_remove_html, do_preprocess=False)
print_words_freq_stat(tokenized_sents_remove_html)


number of unique tokens 111607
number of tokens occuring only once 53582
fraction of tokens occuring only once: 0.48
least 50 common words and their occurrenceL:
[('Moscovite', 1), ('-yawn-', 1), ('appear).If', 1), ('Particulars', 1), ('Svankmajer', 1), ('Ruka', 1), ('sculpt', 1), ('metacinema', 1), ('1.On', 1), ('Paradice', 1), ('Johnathin', 1), ('invlove', 1), ('11.Having', 1), ('9/11-related', 1), ('sibling?A', 1), ('PIggy', 1), ('7.6/10', 1), ('Commissar', 1), ('polution', 1), ('employes', 1), ('inflight', 1), ('MacAdams', 1), ("sensitivity'", 1), ('ubiqutous', 1), ('concept!Fans', 1), ('Aquatania', 1), ('décors', 1), ('Balu', 1), ('Amazons', 1), ('BESIDES', 1), ('Laughtracks', 1), ('Development).This', 1), ('Benecio', 1), ('screening----talk', 1), ("Argentine',this", 1), ('Horror_Fan', 1), ('Mandatory', 1), ('VACATE', 1), ('BUILDING', 1), ('scareless', 1), ('positives-', 1), ('4-D', 1), ('spacewalk', 1), ('moonwalk', 1), ('Mayble', 1), ('Riget3', 1), ('dreams?I', 1), ('expect-', 1

Although we succesfully remove some ugly tokens like `'/>Jules'`, it doesn't statistically improve our dataset. We still have about a half of unique tokens appear only once. It's possible that not too many samples have HTML tags so removing or not removing them does not mattter.

See, sometimes even simple and standard approach on text preprocessing is not very useful. However, there are some tokens that we can do a bit more on cleaning;
- `screening----talk`
- `dreams?I`
- `concept!Fans`

We can safely remove `-` since it doesn't seem to convey meaning (It may be a meaningful feature for some other problems, but not this IMDB review dataset). We can also remove some similar symbols e.g. `,`, `_`, `?`. However, we are not sure if we should remove `!` because it maybe useful feature for this task. Again, these are all gut feelings and not neccesary true.

In [6]:
sents_remove_html_remove_sym = [remove_some_symbols(sent) for sent in sents_remove_html]
tokenized_sents_remove_html_remove_sym = spacy_tokenizer(sents_remove_html_remove_sym)
print_words_freq_stat(tokenized_sents_remove_html_remove_sym)


number of unique tokens 97012
number of tokens occuring only once 39967
fraction of tokens occuring only once: 0.41
least 50 common words and their occurrenceL:
[('HULCE', 1), ('studder', 1), ('eratic', 1), ('belieavablitly', 1), ('Muetos', 1), ('disqualify', 1), ('buzzed', 1), ('of****stars', 1), ('Household', 1), ('Stardom', 1), ('hermiting', 1), ('Jahfre', 1), ('Fondas', 1), ('Safans', 1), ('Roadblock', 1), ('Nekojiru', 1), ('Crockzilla', 1), ('bypassing', 1), ('Moscovite', 1), ('Particulars', 1), ('Svankmajer', 1), ('Ruka', 1), ('sculpt', 1), ('metacinema', 1), ('Paradice', 1), ('Johnathin', 1), ('invlove', 1), ('PIggy', 1), ('Commissar', 1), ('polution', 1), ('employes', 1), ('inflight', 1), ('MacAdams', 1), ('ubiqutous', 1), ('Aquatania', 1), ('décors', 1), ('Balu', 1), ('Amazons', 1), ('BESIDES', 1), ('Laughtracks', 1), ('Benecio', 1), ('Mandatory', 1), ('VACATE', 1), ('BUILDING', 1), ('scareless', 1), ('spacewalk', 1), ('moonwalk', 1), ('Mayble', 1), ('Riget3', 1), ('slajps', 1

Now we get significant improvement. Obviously we can do it better, as we can see we can fix `of****stars` by removing `*`, but we just stop it for now. Don't spend too much time on this!

# Lastly, see how long samples are.

Let's see how long each samples are.

In [7]:
print("average number of tokens per sample %0.2f" % 
      (sum(len(s) for s in tokenized_sents_remove_html_remove_sym) / len(tokenized_sents_remove_html_remove_sym))
)

print("max number of tokens per sample %0.2f" % 
      (max(len(s) for s in tokenized_sents_remove_html_remove_sym))
)

print("min number of tokens per sample %0.2f" % 
      (min(len(s) for s in tokenized_sents_remove_html_remove_sym))
)

average number of tokens per sample 268.07
max number of tokens per sample 2808.00
min number of tokens per sample 10.00


You may be wondering why this is important. We will talk abou this in later experiments.